# Определение перспективного тарифа для телеком компании

---

## Описание проекта

Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана:<br>

«Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.<br>

Вам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. <br>

В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

---

## Описание тарифов

### Тариф «Смарт»
---
1. Ежемесячная плата: 550 рублей<br>
2. Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика<br>
3. Стоимость услуг сверх тарифного пакета:<br>
---
- минута разговора: 3 рубля<br>
- сообщение: 3 рубля<br>
- 1 Гб интернет-трафика: 200 рублей<br>
---

### Тариф «Ультра»
---
1. Ежемесячная плата: 1950 рублей<br>
2. Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика<br>
3. Стоимость услуг сверх тарифного пакета:<br>
---
- минута разговора: 1 рубль<br>
- сообщение: 1 рубль<br>
- 1 Гб интернет-трафика: 150 рублей<br>
---

### Дополнительная информация

«Мегалайн» **всегда округляет вверх** значения минут и мегабайтов. Если пользователь проговорил всего **1 секунду**, в тарифе засчитывается **целая минута**.

## Описание данных
---

Таблица `users` (информация о пользователях):

**user_id** — уникальный идентификатор пользователя<br>
**first_name** — имя пользователя<br>
**last_name** — фамилия пользователя<br>
**age** — возраст пользователя (годы)<br>
**reg_date** — дата подключения тарифа (день, месяц, год)<br>
**churn_date** — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных)<br>
**city** — город проживания пользователя<br>
**tariff** — название тарифного плана<br>

---

Таблица `calls` (информация о звонках):<br>

**id** — уникальный номер звонка<br>
**call_date** — дата звонка<br>
**duration** — длительность звонка в минутах<br>
**user_id** — идентификатор пользователя, сделавшего звонок<br>

---

Таблица `messages` (информация о сообщениях):<br>

**id** — уникальный номер сообщения<br>
**message_date** — дата сообщения<br>
**user_id** — идентификатор пользователя, отправившего сообщение<br>

---

Таблица `internet` (информация об интернет-сессиях):<br>

**id** — уникальный номер сессии<br>
**mb_used** — объём потраченного за сессию интернет-трафика (в мегабайтах)<br>
**session_date** — дата интернет-сессии<br>
**user_id** — идентификатор пользователя<br>

---

Таблица `tariffs` (информация о тарифах):<br>

**tariff_name** — название тарифа<br>
**rub_monthly_fee** — ежемесячная абонентская плата в рублях<br>
**minutes_included** — количество минут разговора в месяц, включённых в абонентскую плату<br>
**messages_included** — количество сообщений в месяц, включённых в абонентскую плату<br>
**mb_per_month_included** — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)<br>
**rub_per_minute** — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)<br>
**rub_per_message** — стоимость отправки сообщения сверх тарифного пакета<br>
**rub_per_gb** — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)<br>

---

## Оглавление

* [1. Открываем файлы с данными и изучаем общую информацию.](#one)
* [2. Предобработка данных.](#two)
* [3. Добавляем новые столбцы.](#three)
* [4. Анализ данных.](#four)
* [5. Проверка гипотез.](#five)
* [Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различается.](#six)
* [Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.](#seven)
* [6. Общие выводы.](#eight)

---

## Шаг 1. Открываем файлы с данными и изучаем общую информацию. <a class="anchor" id="one"></a>

Откроем файлы с данными. Посмотрим на первые тридцать строк методом `head()`, последние тридцать строк методом `tail()` и изучим столбцы методом `info()`

Импортируем необходимые для проекта библиотеки

In [ ]:
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from scipy import stats as st

In [ ]:
# Откроем файлы

tariffs = pd.read_csv('datasets/3_tariffs.csv')
users = pd.read_csv('datasets/3_users.csv')
calls = pd.read_csv('datasets/3_calls.csv')
internet = pd.read_csv('datasets/3_internet.csv')
messages = pd.read_csv('datasets/3_messages.csv')

In [ ]:
def info_data(data):
    display(data.head(30))
    display(data.tail(30))
    data.info()
    print('\n')

print('Информация о тарифах')
info_data(tariffs)
print('Информация о пользователях')
info_data(users)
print('Информация о звонках')
info_data(calls)
print('Информация об интернет-сессиях')
info_data(internet)
print('Информация о сообщениях')
info_data(messages)

---

### Выводы

Прочитали данные из пяти таблиц:

**Тарифы:**

Смарт - недорогой тариф, при этом цена за услуги сверх тарифа дороже, чем в тарифе Ультра.

**Пользователи:**

Информация о 500 пользователях представлена в 8 столбцах.
Столбец `churn_date` преимущественно пуст.
`churn_date` и `reg_date` в будущем переведем в *datetime*.

**Звонки:**

В столбце `duration` много нулевых значений.
Столбец `call_date` переведем в datetime.

**Интернет сессии:**

Столбец `session_date` переведем в datetime.
Столбец `Unnamed: 0` ворвался из ниоткуда, в описании от Заказчика его не было. Поэтому дропнем его.

**Сообщения:**

Столбец `message_date` переведем в datetime.

Пропущенных значений всего 464, это вообще законно?

---

## Шаг 2. Предобработка данных. <a class="anchor" id="two"></a>

Проверим наличие дубликатов в наших таблицах:

In [ ]:
for data in [tariffs, users, calls, internet, messages]:
    print(data.duplicated().sum())

Дубликатов не обнаружено. Удалим ненужный столбец в таблице Интернет сессии.

In [ ]:
internet = internet.drop('Unnamed: 0', axis=1)

В столбце `churn_date` множество пропущенных значений, которые говорят о том прекратил ли клиент пользоваться тарифом или нет, эту информацию в дальнейшем мы не будем использовать.

Переведем в datetime столбцы 'reg_date', 'call_date', 'session_date', 'message_date', год нигде меняться не будет, а в дальнейших расчетах мы будем использовать только месяц. Поэтому оставим только месяц, и для дальнейших объединений переименуем его в `month`.

In [ ]:
# Произведем преобразования в таблице users
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d')
users['reg_date'] = users['reg_date'].dt.month
users = users.rename(columns={'reg_date':'month'})
print('users')
display(users)

# Произведем преобразования в таблице calls
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')
calls['call_date'] = calls['call_date'].dt.month
calls = calls.rename(columns={'call_date':'month'})
print('calls')
display(calls)

# Произведем преобразования в таблице messages
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')
messages['message_date'] = messages['message_date'].dt.month
messages = messages.rename(columns={'message_date':'month'})
print('messages')
display(messages)

# Произведем преобразования в таблице internet
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d')
internet['session_date'] = internet['session_date'].dt.month
internet = internet.rename(columns={'session_date':'month'})
print('internet')
display(internet)

Посмотрим на каждый столбец таблицы users

In [ ]:
def view_data(data):
    for column in data.columns:
        print(column, '\n')
        print(data[column].describe(), '\n')

view_data(users)

В дальнейшим мы будем работать с городами, поэтому надо убедиться, что нет дублей.

In [ ]:
users['city'].unique()

Дубли не обнаружены.

Посмотрим на каждый столбец датасета `calls`

In [ ]:
view_data(calls)

Посчитаем количество пропущенных значений в столбце `duration`

In [ ]:
calls.query('duration == 0').count()

39613 строк! Это много, при этом они не приносят денег и только будут портить всю статистику, поэтому их удалим.

In [ ]:
calls = calls.query('duration != 0')

Как мы помним провайдер округляет минуты в большую сторону, выполним это арифметическое действие

In [ ]:
import numpy as np

In [ ]:
def ceil_row(row):
    row = np.ceil(row)
    return int(row)
calls['duration'] = calls['duration'].apply(ceil_row)
calls_need = calls[['user_id', 'month', 'duration']]

Посмотрим на каждый столбец датасета `internet`

In [ ]:
view_data(internet)

Вновь столкнулись с нулевыми значениями, правда уже в столбце `mb_used`. Мы ведь понимаем, что денег за такие сессии провайдер не получит, поэтому избавимся от них.

In [ ]:
print(internet.query('mb_used == 0').count())
internet = internet.query('mb_used != 0')

Округлим в большую сторону мегабайты

In [ ]:
internet['mb_used'] = internet['mb_used'].apply(ceil_row)
internet

Посмотрим на каждый столбец датасета `messages`

In [ ]:
view_data(messages)

Для удобства в дальнейших вычислениях сделаем сводную таблицу по интересующим нас столбцам

In [ ]:
# в дальнейшем из таблицы users нам понадобятся только столбцы `user_id`, `city`, `tariff`
megaline = users[['user_id', 'city', 'tariff']]

# Добавим категорию городов Москва и Регион
def city_kind(data):
    row = data['city']
    if row == 'Москва':
        return 'Москва'
    return 'Регион'

# Добавим категорию в датасет и исключим city
megaline['city_kind'] = megaline.apply(city_kind, axis=1)
megaline = megaline.drop('city', axis=1)

megaline

Для удобства объединения таблиц в будущем переименуем `tariff_name` в `tariff`

In [ ]:
tariffs = tariffs.rename(columns={'tariff_name':'tariff'})

### Выводы

В запланированных столбцах изменили данные на `datetime` и затем преобразовали их в месяц.

В датасете `calls` удалили звонки, которые длились 0 минут 0 секунд.

В датасете `internet` удалили строки с трафиком 0 мбайт.

В стобце `duration` и `mb_used` округлили значения, чтобы считать в соответствии с тарифными планами оператора.

Создали датасет `megaline`, в котором выделили нужные столбцы из датасета `users`.

Города в новом датасете разделили на две категории `Москва` и `Регион`.

В дальнейшей работе новые столбцы будем добавлять в новый датасет.


---


## Шаг 3. Добавляем новые столбцы. <a class="anchor" id="three"></a>

Количество сделанных звонков и израсходованных минут разговора по месяцам

In [ ]:
calls_need = calls_need.sort_values(by=['user_id', 'month']).groupby(['user_id', 'month']).agg({'duration':['count', 'sum']})
calls_need.columns = ['calls_count', 'calls_sum']
calls_need

Количество отправленных сообщений по месяцам

In [ ]:
messages_need = messages.sort_values(by=['user_id', 'month']).groupby(['user_id', 'month']).agg({'id':'count'})
messages_need.columns = ['messages_count']
messages_need

Объем израсходованного интернет-трафика по месяцам

In [ ]:
internet_need = internet.sort_values(by=['user_id', 'month']).groupby(['user_id', 'month']).agg({'mb_used':'sum'})
internet_need.columns = ['mb_sum']
internet_need

Помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану).

Объединим интересующие нас таблицы

In [ ]:
megaline_use = calls_need.merge(messages_need, on=['user_id', 'month'], how='outer')
megaline_use = megaline_use.merge(internet_need, on=['user_id', 'month'], how='outer')
megaline = megaline_use.reset_index().merge(megaline, on=['user_id'], how='left')
megaline = megaline.sort_values(by=['user_id', 'month'])
megaline = megaline.merge(tariffs, on='tariff', how='left')

In [ ]:
megaline

In [ ]:
f'Количество пользователей: {megaline.user_id.nunique()}'

У нас получились пропущенные значение в результате слияние. В данных таблицах не могли оказаться случайные пропущенные значения и в нашем случае это говорит о том, что пользователь не использовал услугу. Поэтому заменим пропущенные значения на нули.

In [ ]:
megaline = megaline.fillna(0)

In [ ]:
megaline.info()

Напишем функцию, которая принимает датасет и вычисляет выручку.

In [ ]:
def proceeds(data):
    proceeds = data['rub_monthly_fee']
    if data['calls_sum'] > data['minutes_included']:
        proceeds += (data['calls_sum'] - data['minutes_included']) * data['rub_per_minute']
    if data['messages_count'] > data['messages_included']:
        proceeds += (data['messages_count'] - data['messages_included']) * data['rub_per_message']
    if data['mb_sum'] > data['mb_per_month_included']:
        proceeds += np.ceil(((data['mb_sum'] - data['mb_per_month_included']) / 1024)) * data['rub_per_gb']
    return proceeds

Добавим новый столбец `proceeds` с выручкой по каждому пользователю в каждый месяц

In [ ]:
megaline['proceeds'] = megaline.apply(proceeds, axis=1)

Из нашего датасета удалим столбцы с информацией о тарифах, которые в дальнейшем не будем использовать.

In [ ]:
megaline = megaline.drop(['messages_included', 'mb_per_month_included', 'minutes_included', 'rub_monthly_fee',\
                          'rub_per_gb', 'rub_per_message', 'rub_per_minute'], axis=1)

In [ ]:
megaline

### Выводы

В итоговую таблицу `megaline` вычислили и добавили для каждого пользователя по месяцам следующие значения: 

* `calls_count` - количество сделанных звонков
* `calls_sum` - количество израсходованных минут разговора
* `messages_count` - количество отправленных сообщений
* `mb_sum` - объем израсходованного интернет-трафика
* `proceeds` - выручку


---


## Шаг 4. Анализ данных. <a class="anchor" id="four"></a>

Для параметров `calls_sum`, `messages_count` и `mb_sum` посчитаем среднее значение и стандартное отклонение. Дисперсия это квадрат стандартного отклонения, поэтому отдельно его считать не будем.

In [ ]:
report = megaline.pivot_table(index = ['user_id', 'month'],
                             values = ['calls_sum', 'messages_count', 'mb_sum', 'proceeds'],
                             columns = ['tariff'])

# swaplevel(0, 1, axis=1).sort_index(axis=1) задает правильный порядок названий в столбцах

report2 = report.describe().swaplevel(0, 1, axis=1).sort_index(axis=1)
report2.loc[['mean', 'std']].T

In [ ]:
report2.info()

В цикле выведем графики распределения `calls_sum`, `messages_count` и `mb_sum` для каждого тарифа.

In [ ]:
for element in ['calls_sum', 'messages_count', 'mb_sum']:
    for tariff_name in megaline['tariff'].unique():
        (megaline.query('tariff == @tariff_name')[element].hist(bins=10, alpha = 0.5, label = tariff_name))
    plt.title(element)
    plt.legend()
    plt.show()

### Выводы

Для значений `calls_sum`, `messages_count` и `mb_sum` по каждому тарифу посчитали:

* Среднее значение
* Стандартное отклонение

Средние значение в тарифе `ultra`, больше средних в `smart`.
Стандартное отклонение в `ultra` также больше, что говорит о больших расбросах, которые мы увидили на графиках.

Кол-во сообщений в тарифах распределены геометрически.

Графики кол-во израсходованных минут разговора и объем израсходованного интернет-трафика по тарифам соответствуют распределению Пуассона.

По площадям графиков видно, что тариф `smart` более популярный.


---


## Шаг 5. Проверка гипотез <a class="anchor" id="five"></a>

### Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различается. <a class="anchor" id="six"></a>

Сформируем нулевую гипотезу - средняя выручка пользователей тарифов «Ультра» и «Смарт» одинакова.
Альтернативная гипотеза - выручки не равны.
Показатель alpha примем равным 0.05.

Построим гистограмму распределения выручки по каждому тарифу:

In [ ]:
for tariff_name in megaline['tariff'].unique():
    (megaline.query('tariff == @tariff_name')['proceeds'].hist(bins=10, alpha = 0.5, label = tariff_name))
    plt.title('proceeds')
    plt.legend()

Проверим нулевую гипотезу с помощью ттеста:

In [ ]:
mean_proceeds = st.ttest_ind(
    megaline.query('tariff == "smart"')['proceeds'],
    megaline.query('tariff == "ultra"')['proceeds'])

alpha = 0.05

In [ ]:
print('p-значение:', mean_proceeds.pvalue)

if mean_proceeds.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Т.к. мы отвергли нулевую гипотезу, то принимаем альтернативную - выручки не равны.

### Выводы

Мы приняли альтернативную гипотезу - Средняя выручка пользователей тарифов "Ультра" и "Смарт" не одинакова.


---


### Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов. <a class="anchor" id="seven"></a>

Составим нулевую гипотезу - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов. Альтернативная гипотеза - выручки не равны.

Сформируем выборки выручек по Москве и Регионам.

In [ ]:
proceed_moscow = megaline[['city_kind', 'proceeds']].query('city_kind == "Москва"').dropna(subset = ['proceeds'])
proceed_moscow

In [ ]:
proceed_region = megaline[['city_kind', 'proceeds']].query('city_kind == "Регион"').dropna(subset = ['proceeds'])
proceed_region

Построи гистограмму распределения по каждому региону:

In [ ]:
(proceed_region['proceeds'].hist(bins=10, alpha = 0.5, label = 'Регион'))
plt.title('proceeds')
plt.legend()
(proceed_moscow['proceeds'].hist(bins=10, alpha = 0.5, label = 'Москва'))
plt.title('proceeds')
plt.legend()

С помощью ттеста проверим нулевую гипотезу.

In [ ]:
mean_proceeds_region = st.ttest_ind(
    proceed_moscow['proceeds'],
    proceed_region['proceeds'])

alpha = 0.05

In [ ]:
print('p-значение:', mean_proceeds_region.pvalue)

if mean_proceeds_region.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

### Выводы

Приняли нулевую гипотезу - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов.


---


## Шаг 6. Общие выводы <a class="anchor" id="eight"></a>

В ходе работы прочитали данные из пяти таблиц:

**Тарифы:**
**Пользователи:**
**Звонки:**
**Интернет сессии:**
**Сообщения:**

Увидили артефакт в виде дополнительного столбца, который удалили.

Выполнили преобразование данных в столбцах и изменили тип.

Поделили города на две категории: `Москва` и `Регион`.

Создали сводную таблицу `megaline`, в которую добавили следующие значения: 

* `calls_count` - количество сделанных звонков
* `calls_sum` - количество израсходованных минут разговора
* `messages_count` - количество отправленных сообщений
* `mb_sum` - объем израсходованного интернет-трафика
* `proceeds` - выручка

Для значений `calls_sum`, `messages_count` и `mb_sum` по каждому тарифу посчитали:

* Среднее значение
* Стандартное отклонение

Средние значение в тарифе `ultra`, больше средних в `smart`.
Стандартное отклонение в `ultra` также больше, что говорит о больших расбросах, которые мы увидили на графиках.

Кол-во сообщений в тарифах распределены геометрически.

Графики кол-во израсходованных минут разговора и объем израсходованного интернет-трафика по тарифам соответствуют распределению Пуассона.

По площадям графиков видно, что тариф `smart` более популярный.

Мы приняли альтернативную гипотезу - Средняя выручка пользователей тарифов "Ультра" и "Смарт" не одинакова.

Приняли нулевую гипотезу - средняя выручка пользователей из Москвы равна выручке пользователей из других регионов.

Проанализировав поведение клиентов и сделаем выводы — что тариф "Смарт" лучше, т.к. средняя выручка одинакова с тарифом "Ультра", при этом популярность тарифа "Смарт" гораздо выше.

Чтобы скорректировать рекламный бюджет, рекомендуем коммерческому департаменту обратить внимание на тариф "Смарт".


---


### Чек-лист готовности проекта

- [x]  открыты файлы
- [x]  файлы изучены (выведены первые строки, метод info())
- [x]  определены пропущенные значения
- [x]  заполнены пропущенные значения
- [x]  есть пояснение, какие пропущенные значения обнаружены
- [x]  изменены типы данных
- [x]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  посчитано и добавлено в таблицу: количество сделанных звонков и израсходованных минут разговора по месяцам
- [x]  посчитано и добавлено в таблицу: количество отправленных сообщений по месяцам
- [x]  посчитано и добавлено в таблицу: объем израсходованного интернет-трафика по месяцам
- [x]  посчитано и добавлено в таблицу: помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану).
- [x]  построены гистограммы для каждого параметра
- [x]  выполнено задание: Опишите поведение клиентов оператора, исходя из выборки. Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц? Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.
- [x]  проверена гипотеза: средняя выручка пользователей тарифов «Ультра» и «Смарт» различается;
- [x]  проверенеа гипотеза: средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов
- [x]  выполнено задание: Пороговое значение alpha задано
- [x]  пояснено: как вы формулировали нулевую и альтернативную гипотезы
- [x]  пояснено: какой критерий использовали для проверки гипотез и почему.
- [x]  в каждом этапе есть выводы
- [x]  есть общий вывод